In [147]:
from __future__ import print_function

import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

In [148]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('calendar', 'v3', credentials=creds)

    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                            singleEvents=True, maxResults=100,
                                            orderBy='startTime').execute()
    events = events_result.get('items')

    if not events:
        print('No upcoming events found.')

except HttpError as error:
    print('An error occurred: %s' % error)


Getting the upcoming 10 events


In [149]:
now = datetime.datetime.utcnow()
now

datetime.datetime(2022, 6, 16, 14, 32, 24, 369412)

In [150]:
now + datetime.timedelta(days=1)

datetime.datetime(2022, 6, 17, 14, 32, 24, 369412)

In [151]:
# Prints the start and name of the next 10 events   
color = {
    None:   "🐬",
    "1":    "🌊",
    "2":    "🗽",
    "3":    "🍇",
    "4":    "🦜",
    "5":    "🐆",
    "6":    "🦀",
    "7":    "🌍",
    "8":    "🐘",
    "9":    "🏞️",
    "10":   "🐢",
    "11":   "🍎",
}

now = datetime.datetime.strptime(events[k]['start']['date'],"%Y-%m-%d")
for k,e in enumerate(events):
    if 'date' in e['start'].keys():
        events[k]['start']['date'] = datetime.datetime.strptime(events[k]['start']['date'],"%Y-%m-%d")
        events[k]['end']['date'] = datetime.datetime.strptime(events[k]['end']['date'],"%Y-%m-%d")
        
        events[k]['start']['date'] = events[k]['start']["date"].strftime("%d/%m/%Y")
        events[k]['start']['time'] = "All Day"
        events[k]['end']['date'] = events[k]['end']["date"].strftime("%d/%m/%Y")
        events[k]['end']['time'] = "All Day"
        
    elif 'dateTime' in e['start'].keys():     
        events[k]['start']['dateTime'] = datetime.datetime.strptime(events[k]['start']['dateTime'],"%Y-%m-%dT%H:%M:%S+%U:%W")
        events[k]['end']['dateTime'] = datetime.datetime.strptime(events[k]['end']['dateTime'],"%Y-%m-%dT%H:%M:%S+%U:%W")
        
        events[k]['start']['date'] = events[k]['start']["dateTime"].strftime("%d/%m/%Y")
        events[k]['start']['time'] = events[k]['start']["dateTime"].strftime("%H:%M")
        events[k]['end']['date'] = events[k]['end']["dateTime"].strftime("%d/%m/%Y")
        events[k]['end']['time'] = events[k]['end']["dateTime"].strftime("%H:%M")
        
for event in events: 
    print(event['start']['date'], event['start']['time'], "->", event['end']['date'], event['end']['time'], event['summary'])

17/06/2022 13:30 -> 17/06/2022 15:30 EXAM: Sensors from h13.30
18/06/2022 All Day -> 19/06/2022 All Day Ghent
21/06/2022 11:50 -> 21/06/2022 13:25 Flight to Milan (FR 4526)
22/06/2022 All Day -> 23/06/2022 All Day Meeting NearLab - Ke
30/06/2022 All Day -> 01/07/2022 All Day Neuroengineering Exam 1st call
30/06/2022 10:00 -> 30/06/2022 12:00 Esame MAM Ventu
01/07/2022 All Day -> 02/07/2022 All Day Computational Biomechanics Exam 1st call
05/07/2022 All Day -> 06/07/2022 All Day Eliminazione Instagram
18/07/2022 All Day -> 19/07/2022 All Day Computational Biomechanics Exam 2nd call
20/07/2022 All Day -> 21/07/2022 All Day Laurea Ventu
21/07/2022 All Day -> 22/07/2022 All Day Neuroengineering Exam 2nd call
26/07/2022 All Day -> 27/07/2022 All Day Biosensors Exam 2nd call


In [152]:
color[None]

'🐬'

In [168]:
from telegram.ext import (Updater, 
    CommandHandler,
    MessageHandler,
    CallbackQueryHandler,
    ConversationHandler,
    CallbackContext,    
    Filters
)
import datetime
with open("telegramapi.txt") as f:
        tokens = f.readlines()
bot_token = tokens[0].strip("\n")
updater = Updater(token=bot_token, use_context=True)
        

        
print(">> "+datetime.datetime.now().strftime("[%d/%m/%Y-%H:%M:%S]-BOT: ")+
        " Loading Telegram bot: ",end="")
dispatcher = updater.dispatcher
print("SUCCESS") if updater!=0 else print("FAIL")

def send_message(context):
    user_id = 410773404
    context.bot.send_message(
        chat_id=user_id,
        text="Enculet!"
    )

def printid(update, context):
    userid = update.message.from_user['id']
    print(userid)
    context.bot.send_message(
        chat_id=update.effective_chat.id, 
        text=userid
    )


# dispatcher.add_handler(CommandHandler('send_message', send_message))
# dispatcher.add_handler(CommandHandler('start', printid))

# jobqueue = updater.job_queue
# jobqueue.run_repeating(
#     callback=send_message,
#     interval=1,
# )

def center(text, w,sep='_'):
    ext = sep
    left = (w-len(text))//2
    right = w-left-len(text)
    return left*ext+ text + right*ext

MENU = 0
LINEWIDTH = 36
BT = "`"
SEP = "_"
import telegram
from telegram import ReplyKeyboardMarkup
from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update


def events_callback(update, context):
    user_id = update.message.from_user['id']
    menu_options = [
        ['Events','Tasks']
    ]
    
    for e in events:
        events_keyboard = []
        print(e['summary'])
        if datetime.datetime.utcnow().strftime("%d/%m/%Y") == e['start']['date']:
            update.message.reply_text(
                BT+"*"+center("TODAY",LINEWIDTH,sep=" ")+"*"+BT+"\n"+center("-",LINEWIDTH,sep = "-")+BT,
                parse_mode=telegram.ParseMode.MARKDOWN
            )
        elif (datetime.datetime.utcnow()+datetime.timedelta(days=1)).strftime("%d/%m/%Y") == e['start']['date']:
            update.message.reply_text(                
                "*"+center("TOMORROW",LINEWIDTH,sep=" ")+"*"+"\n"+BT+center("-",LINEWIDTH,sep = "-")+BT,
                parse_mode=telegram.ParseMode.MARKDOWN
            )
        else:
            update.message.reply_text(                
                "*"+center("FURTHER",LINEWIDTH,sep=" ")+"*"+"\n"+BT+center("-",LINEWIDTH,sep = "-")+BT,
                parse_mode=telegram.ParseMode.MARKDOWN
            )
        # One day events
        if e['start']['date'] == e['end']['date']:
            
            # All day events
            if e['start']['time'] == "All Day":
                events_keyboard.append([
                        InlineKeyboardButton(color[e['colorId']]+" "+e['summary']+" "+color[e['colorId']], callback_data="f")
                ])
                msg_text = e['start']['date']+"  All Day"
            else:
                events_keyboard.append([
                        InlineKeyboardButton(color[e['colorId']]+" "+e['summary']+" "+color[e['colorId']], callback_data="f")
                ])
                msg_text = e['start']['date']+"-"+e['start']['time']+" ➡️ "+ e['end']['date']+"-"+e['end']['time']
                
        # Multiple day events
        else:
            events_keyboard.append([
                    InlineKeyboardButton(color[e['colorId']]+" "+e['summary']+" "+color[e['colorId']], callback_data="f")
            ])
            msg_text = e['start']['date']+" ➡️ "+ e['end']['date']
        reply_markup = InlineKeyboardMarkup(events_keyboard,resize_keyboard=True)
        
        update.message.reply_text(
            BT+center(msg_text,LINEWIDTH)+BT,
            parse_mode=telegram.ParseMode.MARKDOWN,
            reply_markup = reply_markup
        )
    return MENU

def tasks_callback(update,context):
    client=context.dispatcher.user_data['client']
    user_id = update.message.from_user['id']
    menu_options = [
        ['Events','Tasks']
    ]

    update.message.reply_text(
        text="Showing Tasks NOW",
        # chat_id=update.effective_chat.id,
        parse_mode=telegram.ParseMode.HTML,
        reply_markup=ReplyKeyboardMarkup(
            menu_options, 
            one_time_keyboard=True,
            resize_keyboard=True,
            )
    )
    return MENU

def menu(update, context):
    user_id = update.message.from_user['id']
    print(">> "+datetime.datetime.now().strftime(f"[%d/%m/%Y-%H:%M:%S]-{user_id}: ")+
        " MENU request ")
    
    menu_options = [
        ['Events','Tasks']
    ]

    update.message.reply_text(
        "Select from the menu:  ",
        reply_markup=ReplyKeyboardMarkup(
            menu_options, 
            one_time_keyboard=True,
            resize_keyboard=True,
        ),
    )
    return MENU

conv_handler = ConversationHandler(
        entry_points=[CommandHandler('menu',menu)],
        states={
            MENU: 
                [
                MessageHandler(Filters.regex('^(Events)$'), events_callback),
                MessageHandler(Filters.regex('^(Tasks)$'), tasks_callback),
                ],
        },
        fallbacks=[CommandHandler('menu', menu)]
)
dispatcher.add_handler(conv_handler)
dispatcher.add_handler(CommandHandler('menu', menu))

updater.start_polling()
updater.idle()

>> [16/06/2022-17:01:29]-BOT:  Loading Telegram bot: SUCCESS
>> [16/06/2022-17:01:31]-410773404:  MENU request 
EXAM: Sensors from h13.30
Ghent
Flight to Milan (FR 4526)
Meeting NearLab - Ke
Neuroengineering Exam 1st call
Esame MAM Ventu
Computational Biomechanics Exam 1st call
Eliminazione Instagram
Computational Biomechanics Exam 2nd call
Laurea Ventu


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\Users\alber\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "c:\Users\alber\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\conversationhandler.py", line 626, in handle_update
    new_state = handler.handle_update(update, dispatcher, check_result, context)
  File "c:\Users\alber\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\handler.py", line 198, in handle_update
    return self.callback(update, context)
  File "C:\Users\alber\AppData\Local\Temp/ipykernel_19236/2572307405.py", line 99, in events_callback
    InlineKeyboardButton(color[e['colorId']]+" "+e['summary']+" "+color[e['colorId']], callback_data="f")
KeyError: 'colorId'


In [ ]:
type("helo")

str

In [ ]:
type(events[6]['id'])

str

In [ ]:
events_keyboard = []
for e in events:
    print(e['summary'])
    if 'date' in e['start'].keys():
        events_keyboard.append([
            InlineKeyboardButton(e['summary'], 
                                    callback_data= "f")]
        )
    elif 'dateTime' in e['start'].keys():
        events_keyboard.append([
            InlineKeyboardButton(e['summary'], 
                                    callback_data= "f")]
        )
events_keyboard[0]

EXAM: Sensors from h13.30
Ghent
Flight to Milan (FR 4526)
Meeting NearLab - Ke
Neuroengineering Exam 1st call
Esame MAM Ventu
Computational Biomechanics Exam 1st call
Eliminazione Instagram
Computational Biomechanics Exam 2nd call
Laurea Ventu
Neuroengineering Exam 2nd call
Biosensors Exam 2nd call
